<a href="https://colab.research.google.com/github/rk119/F20BC/blob/main/Copy_of_F20BC_CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Imports

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

### Load the data

In [3]:
df  = pd.read_csv("/content/drive/MyDrive/F20BC-Dataset/data_banknote_authentication.csv")

### View the first 5 instances of the data

In [4]:
df.head()

,3.6216,8.6661,-2.8073,-0.44699,0
0,4.54590,8.1674,-2.4586,-1.46210,0
1,3.86600,-2.6383,1.9242,0.10645,0
2,3.45660,9.5228,-4.0112,-3.59440,0
3,0.32924,-4.4552,4.5718,-0.98880,0
4,4.36840,9.6718,-3.9606,-3.16250,0


In [5]:
df.shape

(1371, 5)

In [6]:
# if len(df.columns) == 1:
#     # Split the single column into multiple columns by the comma delimiter
#     df = df.iloc[:, 0].str.split(',', expand=True)

# # Convert the split columns to numeric types
# df = df.apply(pd.to_numeric)

# Check if there are 5 columns
if df.shape[1] == 5:
    # Split into features (X) and label (Y)
    X = df.iloc[:, :-1]
    Y = df.iloc[:, -1]

# Convert X and Y to numpy arrays
X_array = X.to_numpy()
Y_array = Y.to_numpy()

In [7]:
Y

0       0
1       0
2       0
3       0
4       0
       ..
1366    1
1367    1
1368    1
1369    1
1370    1
Name: 0, Length: 1371, dtype: int64

In [8]:
Y_array

array([0, 0, 0, ..., 1, 1, 1])

In [9]:
X

,3.6216,8.6661,-2.8073,-0.44699
0,4.54590,8.16740,-2.4586,-1.46210
1,3.86600,-2.63830,1.9242,0.10645
2,3.45660,9.52280,-4.0112,-3.59440
3,0.32924,-4.45520,4.5718,-0.98880
4,4.36840,9.67180,-3.9606,-3.16250
...,...,...,...,...
1366,0.40614,1.34920,-1.4501,-0.55949
1367,-1.38870,-4.87730,6.4774,0.34179
1368,-3.75030,-13.45860,17.5932,-2.77710
1369,-3.56370,-8.38270,12.3930,-1.28230


In [10]:
X_array

array([[  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645],
       [  3.4566 ,   9.5228 ,  -4.0112 ,  -3.5944 ],
       ...,
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ]])

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [12]:
df_normalized

,3.6216,8.6661,-2.8073,-0.44699,0
0,0.835659,0.820982,0.121804,0.644326,0.0
1,0.786629,0.416648,0.310608,0.786951,0.0
2,0.757105,0.871699,0.054921,0.450440,0.0
3,0.531578,0.348662,0.424662,0.687362,0.0
4,0.822859,0.877275,0.057100,0.489711,0.0
...,...,...,...,...,...
1366,0.537124,0.565855,0.165249,0.726398,1.0
1367,0.407690,0.332868,0.506753,0.808350,1.0
1368,0.237385,0.011768,0.985603,0.524755,1.0
1369,0.250842,0.201701,0.761587,0.660675,1.0


In [13]:
if len(df_normalized.columns) == 1:
    # Split the single column into multiple columns by the comma delimiter
    df_normalized = df_normalized.iloc[:, 0].str.split(',', expand=True)

# Convert the split columns to numeric types
df_normalized = df_normalized.apply(pd.to_numeric)

# Check if there are 5 columns
if df_normalized.shape[1] == 5:
    # Split into features (X) and label (Y)
    X_norm = df_normalized.iloc[:, :-1]
    Y_norm = df_normalized.iloc[:, -1]

# Convert X and Y to numpy arrays
X_array_norm = X_norm.to_numpy()
Y_array_norm = Y_norm.to_numpy()

In [14]:
X_array_norm

array([[0.83565902, 0.82098209, 0.12180412, 0.64432563],
       [0.78662859, 0.41664827, 0.31060805, 0.78695091],
       [0.75710505, 0.87169921, 0.05492063, 0.45043964],
       ...,
       [0.23738543, 0.01176814, 0.98560321, 0.52475518],
       [0.25084193, 0.20170105, 0.76158701, 0.6606745 ],
       [0.32452819, 0.49074676, 0.34334762, 0.88594888]])

In [15]:
len(X_array)

1371

In [16]:
len(Y_array)

1371

# Functions

### Activation Functions

In [17]:
class ActivationFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

class Identity:
  def evaluate(self,x):
    return x
  def derivative(self,x):
    return 1

class Sigmoid(ActivationFunctions):
  def evaluate(self,x):
    return 1 / (1 + np.exp(-x))
  def derivative(self,x):
    f = self.evaluate(x)
    return f * (1-f)

class Tanh(ActivationFunctions):
  def evaluate(self,x):
    return np.tanh(x)
  def derivative(self,x):
    f = self.evaluate(x)
    return 1 - f ** 2

class relu(ActivationFunctions):
  def evaluate(self,x):
    return np.maximum(0, x)
  def derivative(self,x):
    return (x > 0).astype(float)

### Loss Functions

In [18]:
class LossFunctions:
  def evaluate(self,x):
    pass
  def derivate(self,x):
    pass

# y is predictions made by the neural network
# t is target/actual numbers corresponding to inputs
class Mse(LossFunctions):
  def evaluate(self, y, t):
    return ((t - y) ** 2).mean()
  def derivative(self, y, t):
    return 2 * (y - t) / len(y)

class BinaryCrossEntropy(LossFunctions):
  def evaluate(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    term0 = (1 - t) * np.log(1 - y_pred + 1e-7)
    term1 = t * np.log(y_pred + 1e-7)
    return - (term0 + term1).mean()

  def derivative(self, y, t):
    y_pred = np.clip(y, 1e-7, 1 - 1e-7)
    return (t / y_pred) - (1 - t) / (1 - y_pred)

class Hinge(LossFunctions):
  def evaluate(self, y, t):
    return np.maximum(0, 1 - t * y).mean()

  def derivative(self, y, t):
    return np.where(t * y < 1, -t, 0)

# Neural Network

In [136]:
class InputLayer:
    def __init__(self, input_size):
        self.nb_nodes = input_size

    def forward(self, input_data):
        return input_data

class Layer:
    def __init__(self, input_size, nodes, activation, weights=None, biases=None):
        self.nb_nodes = nodes
        self.W = weights if weights is not None else np.random.randn(input_size, nodes)
        self.B = biases if biases is not None else np.random.randn(nodes)
        self.activation = activation

    def forward(self, input_data):
        self.X_in = input_data
        z = np.dot(input_data, self.W) + self.B
        out = self.activation.evaluate(z)
        return out

class NeuralNetwork:
    def __init__(self, configuration, position=None):
        self.layers = []
        input_size = configuration[0]

        # The input layer is simply added as a pass-through layer
        self.layers.append(InputLayer(input_size))

        # If a position vector is provided, it contains weights and biases for each layer
        if position is not None:
            for idx, (nodes, activation) in enumerate(configuration[1:]):
                weights, biases = position[idx]
                # print("layer ", idx, "W = ",weights, "B = ", biases )
                layer = Layer(input_size, nodes, activation, weights=weights, biases=biases)
                self.add(layer)
                input_size = nodes  # Update input size for the next layer
        else:
            # If no position vector, initialize layers with random weights and biases
            for nodes, activation in configuration[1:]:
                layer = Layer(input_size, nodes, activation)
                self.add(layer)
                input_size = nodes  # Update input size for the next layer

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, input_data):
        for layer in self.layers:
            input_data = layer.forward(input_data)
        return input_data

    def print_layers(self):
        for i, layer in enumerate(self.layers):
            if isinstance(layer, InputLayer):
                print(f"Layer {i}: Input Layer with {layer.nb_nodes} nodes")
            elif isinstance(layer, Layer):
                print(f"Layer {i}: Hidden Layer with {layer.nb_nodes} nodes, Activation Function: {layer.activation.__class__.__name__}")
                print(f"  Weights Shape: {layer.W.shape}, Biases Shape: {layer.B.shape}")
                print(f"  Weights= : {layer.W}, Biases= {layer.B}")
            else:
                print(f"Layer {i}: Unknown Layer Type")

    def flatten_weights_and_biases(self):
        flattened_vector = []
        for layer in self.layers:
            if isinstance(layer, Layer):
                # Flatten and append weights and biases of this layer
                flattened_weights = layer.W.flatten()
                flattened_biases = layer.B.flatten()
                flattened_vector.extend(flattened_weights.tolist())
                flattened_vector.extend(flattened_biases.tolist())
        return flattened_vector

In [111]:
def unflatten_weights_and_biases(flattened_vector, configuration):
    position = []
    index = 0

    # Skip the input layer configuration, as it doesn't have weights or biases
    for nodes, _ in configuration[1:]:
        # Previous layer's node count is the number of rows for weights
        input_size = configuration[0] if not position else position[-1][0].shape[1]

        # Calculate the number of weights and biases
        num_weights = input_size * nodes
        num_biases = nodes

        # Extract weights and biases from the flattened vector
        weights = flattened_vector[index : index + num_weights]
        biases = flattened_vector[index + num_weights : index + num_weights + num_biases]

        # Reshape weights to the correct dimensions and add to position
        position.append((np.array(weights).reshape(input_size, nodes), np.array(biases)))

        # Update the index
        index += num_weights + num_biases

    return position

In [112]:
relu_activation = relu()
sigmoid_activation = Sigmoid()
tanh_activation = Tanh()

In [113]:
config = [
    4,
    [3, relu_activation],
    [4, sigmoid_activation],
    [1, sigmoid_activation]
]

In [114]:
nn = NeuralNetwork(config)

In [115]:
nn.print_layers()

Layer 0: Input Layer with 4 nodes
Layer 1: Hidden Layer with 3 nodes, Activation Function: relu
  Weights Shape: (4, 3), Biases Shape: (3,)
  Weights= : [[ 0.08758009 -0.95004972 -0.64814806]
 [-0.35020182  0.56449234  2.11533166]
 [-1.73868223 -1.04293742 -1.61367125]
 [ 0.28645186  0.09103718 -2.69138029]], Biases= [-0.20746207  0.08972316  0.91606226]
Layer 2: Hidden Layer with 4 nodes, Activation Function: Sigmoid
  Weights Shape: (3, 4), Biases Shape: (4,)
  Weights= : [[-0.18089225  0.16979938  1.89086358 -0.69018195]
 [ 0.83558715 -0.26043971  0.7940041   1.33879149]
 [ 1.49390263  0.61500348  0.00230536  2.02432866]], Biases= [ 2.56870324 -0.08161351  0.23762439  1.64214324]
Layer 3: Hidden Layer with 1 nodes, Activation Function: Sigmoid
  Weights Shape: (4, 1), Biases Shape: (1,)
  Weights= : [[-1.33570044]
 [-1.38832104]
 [ 0.20499419]
 [ 0.35174906]], Biases= [0.73447658]


In [116]:
flattened_vector = nn.flatten_weights_and_biases()
print(flattened_vector)

[0.08758009162565487, -0.9500497246199046, -0.6481480570151313, -0.35020182388456705, 0.5644923435932719, 2.1153316563558064, -1.738682225588879, -1.0429374206996673, -1.6136712545796774, 0.28645185706289256, 0.09103717655363429, -2.691380294150482, -0.2074620695521379, 0.08972315965686356, 0.9160622551650549, -0.18089224757533204, 0.1697993827797595, 1.8908635779840406, -0.6901819533917078, 0.8355871502576118, -0.2604397078103472, 0.7940041003008417, 1.3387914911904226, 1.4939026305231276, 0.6150034830308004, 0.0023053607479257645, 2.0243286564176395, 2.5687032355328236, -0.08161350564282391, 0.23762439136692434, 1.6421432380605876, -1.3357004367870777, -1.3883210360648406, 0.20499419091724536, 0.3517490576093228, 0.7344765752813294]


In [118]:
# unflatten_weights_and_biases(flattened_vector, config)

In [119]:
input_data = X_array_norm

nn_init_pred = nn.forward(input_data)

In [120]:
nn_init_pred

array([[0.30853436],
       [0.31806559],
       [0.27108568],
       ...,
       [0.31806559],
       [0.31806559],
       [0.31806559]])

In [121]:
len(nn_init_pred)

1371

In [140]:
def calculate_fitness(neural_network, X, Y, loss_function):
    predictions = neural_network.forward(X)
    # print(predictions[0])
    loss = loss_function.evaluate(predictions.flatten(), Y)
    # print("Loss",loss)
    # print("predictions = ",predictions.flatten())
    # print("len(predictions) = ",len(predictions.flatten()))
    return loss,predictions

loss_function = BinaryCrossEntropy()
initial_fitness,predictions = calculate_fitness(nn, X, Y, loss_function)
print("initial_fitness", initial_fitness)

initial_fitness 0.7711894790593579


In [174]:
def forward_pass(X, Y, W):
    # Rebuild the neural network with the given weights
    position = unflatten_weights_and_biases(W, config)
    # print(position)
    neural_network = NeuralNetwork(config, position=position)

    # Perform forward pass
    predictions = neural_network.forward(X)

    # Calculate loss
    loss = loss_function.evaluate(predictions.flatten(), Y)
    return loss

In [160]:
loss,predictions = forward_pass(X, Y, flattened_vector)
print(loss)

0.7711894790593579


In [161]:
def get_accuracy(Y, Y_pred):
    """
    Calcualtes accuracy.
    :param Y: int(N, )
        Correct labels.
    :param Y_pred: int(N, ) | double(N, C)
        Predicted labels of shape(N, ) or (N, C) in case of one-hot vector.
    :return: double
        Accuracy.
    """
    predicted_classes = (Y_pred >= 0.5).astype(int) # Convert probabilities to class labels
    print(predicted_classes)
    return (Y == predicted_classes).mean()

In [162]:
get_accuracy(Y_array, predictions.flatten())

[0 0 0 ... 0 0 0]


0.5550692924872356

In [163]:
class Particle:
    """
    Particle class represents a solution inside a pool(Swarm).
    """
    def __init__(self, no_dim, x_range, v_range):
        """
        Particle class constructor
        :param no_dim: int
            No of dimensions.
        :param x_range: tuple(double)
            Min and Max value(range) of dimension.
        :param v_range: tuple(double)
            Min and Max value(range) of velocity.
        """
        self.x = np.random.uniform(x_range[0], x_range[1], (no_dim, )) #particle position in each dimension...
        self.v = np.random.uniform(v_range[0], v_range[1], (no_dim, )) #particle velocity in each dimension...
        self.pbest = np.inf
        self.pbestpos = np.zeros((no_dim, ))

In [235]:
class Swarm:
    """
    Swarm class represents a pool of solution(particle).
    """
    def __init__(self, no_particle, no_dim, x_range, v_range, iw_range, c):
        """
        Swarm class constructor.
        :param no_particle: int
            No of particles(solutions).
        :param no_dim: int
            No of dimensions.
        :param x_range: tuple(double)
            Min and Max value(range) of dimension.
        :param v_range: tuple(double)
            Min and Max value(range) of velocity.
        :param iw_range: tuple(double)
            Min and Max value(range) of interia weight.
        :param c: tuple(double)
            c[0] -> cognitive parameter, c[1] -> social parameter.
        """
        self.p = np.array([Particle(no_dim, x_range, v_range) for i in range(no_particle)])
        self.gbest = np.inf
        self.gbestpos = np.zeros((no_dim, ))
        self.x_range = x_range
        self.v_range = v_range
        self.iw_range = iw_range
        self.c0 = c[0]
        self.c1 = c[1]
        self.no_dim = no_dim

    def optimize(self, function, X, Y,  print_step,  iter):
        """
        optimize is used start optimization.

        :param function: function
            Function to be optimized.
        :param X: input
            Used in forward pass.
        :param Y: target
            Used to calculate loss.
        :param print_step: int
            Print pause between two adjacent prints.
        :param iter: int
            No of iterations.
        """
        for i in range(iter):
            for particle in self.p:
                # print("particle.x",particle.x)
                fitness = function(X, Y, particle.x)
                # print("fitness",fitness)
                # print("particle.pbest",particle.pbest)

                if fitness < particle.pbest:
                    particle.pbest = fitness
                    particle.pbestpos = particle.x.copy()

                if fitness < self.gbest:
                    self.gbest = fitness
                    self.gbestpos = particle.x.copy()


            for particle in self.p:
                #Here iw is inertia weight...
                iw = np.random.uniform(self.iw_range[0], self.iw_range[1], 1)[0]
                particle.v = iw * particle.v + (self.c0 * np.random.uniform(0.0, 1.0, (self.no_dim, )) * \
                (particle.pbestpos - particle.x)) + (self.c1 * np.random.uniform(0.0, 1.0, (self.no_dim, )) \
                * (self.gbestpos - particle.x))
                #particle.v = particle.v.clip(min=self.v_range[0], max=self.v_range[1])
                particle.x = particle.x + particle.v
                #particle.x = particle.x.clip(min=self.x_range[0], max=self.x_range[1])

            if i % print_step == 0:
                print('iteration#: ', i+1,  ' loss: ', fitness)

        print("global best loss: ", self.gbest)

    def get_best_solution(self):
        '''

        :return: array of parameters/weights.
        '''
        return self.gbestpos

In [168]:
def get_num_dim(configuration):
  total_weights = 0
  total_biases = 0

  # The number of nodes in the previous layer, initially the input layer
  prev_nodes = configuration[0]

  # Loop through each layer (excluding the input layer)
  for layer in configuration[1:]:
      # Extract the number of nodes in the current layer
      nodes = layer[0]

      # Calculate weights and biases for the current layer
      weights = prev_nodes * nodes
      biases = nodes

      # Add to total weights and biases
      total_weights += weights
      total_biases += biases

      # Update prev_nodes for the next iteration
      prev_nodes = nodes

  # Output the total number of dimensions
  total_dimensions = total_weights + total_biases
  return total_dimensions

In [236]:
if __name__ == '__main__':
    no_solution = 10
    no_dim = get_num_dim(config)
    w_range = (0.0, 1.0)
    lr_range = (0.0, 1.0)
    iw_range = (0.9, 0.9)  # iw -> inertial weight...
    c = (0.5, 0.3)  # c[0] -> cognitive factor, c[1] -> social factor...

    s = Swarm(no_solution, no_dim, w_range, lr_range, iw_range, c)

    # Train:Test Split
    test_size = 0.2  # User defined split ratio
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size, random_state=42)

    s.optimize(forward_pass, X_train, Y_train, 10, 100)
    W = s.get_best_solution()


     # Perform forward pass with the best solution (weights)
    best_position = unflatten_weights_and_biases(W, config)
    best_nn = NeuralNetwork(config, position=best_position)
    # Y_pred = best_nn.forward(X).flatten()

    Y_pred = best_nn.forward(X_test).flatten()

    # Calculate and print accuracy
    accuracy = get_accuracy(Y_test.to_numpy().flatten(), Y_pred)
    print("Accuracy: %.3f" % accuracy)

    # Y_pred =
    # # print("Y_pred",Y_pred)
    # accuracy = get_accuracy(Y, Y_pred)
    # print("Accuracy: %.3f"% accuracy)

iteration#:  1  loss:  1.2486814879695585
iteration#:  11  loss:  0.6515561663884571
iteration#:  21  loss:  0.5015818687703557
iteration#:  31  loss:  0.4478150380019697
iteration#:  41  loss:  0.33958361743336185
iteration#:  51  loss:  0.31418844420545566
iteration#:  61  loss:  0.2390075462256962
iteration#:  71  loss:  0.19679586679641653
iteration#:  81  loss:  0.1455687750606852
iteration#:  91  loss:  0.11394077083397398
global best loss:  0.10158503103072393
[0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0 1
 1 1 0 0 1 1 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 1 1 1 1 0 1 0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 1 1 1
 1 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1
 1 0 1 1 1 0 1 0 1 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 1 0 1
 0 1 1 0 0 1 0 0 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 1 1 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 1 0 0 0 1
 1 

# gitty

In [39]:
from sklearn.datasets import load_iris
import numpy as np

In [40]:
# #load iris dataset..
# data = load_iris()

# #Store input & target in X and Y..
# X = X
# Y = Y

In [41]:
#define no of nodes in each layer..
INPUT_NODES = 4
HIDDEN_NODES = 20
OUTPUT_NODES = 2

In [42]:
def one_hot_encode(Y):
    """
    create one-hot encoded vectors from target labels(Y).
    :param Y: int(N, )
    :return: int(N, C)
        Returns an array of shape(N, C) where C is number of classes.
    """
    num_unique = len(np.unique(np.array(Y)))
    zeros = np.zeros((len(Y), num_unique))
    zeros[range(len(Y)), Y] = 1
    return zeros

In [43]:
def softmax(logits):
    """
    Apply softmax function on logits and return probabilities.
    :param logits: double(N, C)
        Logits of each instance for each class.
    :return: double(N, C)
        probability for each class of each instance.
    """
    exps = np.exp(logits)
    return exps / np.sum(exps, axis=1, keepdims=True)

In [44]:
def Negative_Likelihood(probs, Y):
    """
    Calculates Negative Log Likelihood loss.
    :param probs: double(N, C)
        Probability of each instance for each class.
    :param Y: int(N, )
        Integer representation of each class.
    :return: double
        Returns value of loss calculated.
    """
    num_samples = len(probs)
    corect_logprobs = -np.log(probs[range(num_samples), Y])
    return np.sum(corect_logprobs) / num_samples

def Cross_Entropy(probs, Y):
    """
    Calculates Categorical Cross Entropy loss.
    :param probs: double(N, C)
        Probability of each instance for each class.
    :param Y: int(N, C)
        One-hot encoded representation of classes.
    :return: double
        Returns value of loss calculated.
    """
    num_samples = len(probs)
    ind_loss = np.max(-1 * Y * np.log(probs + 1e-12), axis=1)
    return np.sum(ind_loss) / num_samples

In [63]:
def forward_pass(X, Y, W):
    """
    Performs forward pass during Neural Net training.
    :param X: double(N, F)
        X is input where N is number of instances and F is number of features.
    :param Y: int(N, ) | int(N, C)
        Y is target where N is number of instances and C is number of classes in case of
        one-hot encoded target.
    :param W: double(N, )
        Weights where N is number of total weights(flatten).
    :return: double
        Returns loss of forward pass.
    """

    if isinstance(W, Particle):
        W = W.x

    w1 = W[0 : INPUT_NODES * HIDDEN_NODES].reshape((INPUT_NODES, HIDDEN_NODES))
    b1 = W[INPUT_NODES * HIDDEN_NODES:(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES].reshape((HIDDEN_NODES, ))
    w2 = W[(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES:(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES +\
        (HIDDEN_NODES * OUTPUT_NODES)].reshape((HIDDEN_NODES, OUTPUT_NODES))
    b2 = W[(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES + (HIDDEN_NODES * OUTPUT_NODES): (INPUT_NODES *\
        HIDDEN_NODES) + HIDDEN_NODES + (HIDDEN_NODES * OUTPUT_NODES) + OUTPUT_NODES].reshape((OUTPUT_NODES, ))

    z1 = np.dot(X, w1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, w2) + b2
    logits = z2

    probs = softmax(logits)
    print("Negative_Likelihood(probs, Y)",Negative_Likelihood(probs, Y))
    return Negative_Likelihood(probs, Y)

In [46]:
def predict(X, W):
    """
    Performs forward pass during Neural Net test.
    :param X: double(N, F)
        X is input where N is number of instances and F is number of features.
    :param W: double(N, )
        Weights where N is number of total weights(flatten).
    :return: int(N, )
        Returns predicted classes.
    """

    w1 = W[0: INPUT_NODES * HIDDEN_NODES].reshape((INPUT_NODES, HIDDEN_NODES))
    b1 = W[INPUT_NODES * HIDDEN_NODES:(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES].reshape((HIDDEN_NODES,))
    w2 = W[(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES:(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES + \
        (HIDDEN_NODES * OUTPUT_NODES)].reshape((HIDDEN_NODES, OUTPUT_NODES))
    b2 = W[(INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES + (HIDDEN_NODES * OUTPUT_NODES): (INPUT_NODES * \
        HIDDEN_NODES) + HIDDEN_NODES + (HIDDEN_NODES * OUTPUT_NODES) + OUTPUT_NODES].reshape((OUTPUT_NODES,))

    z1 = np.dot(X, w1) + b1
    a1 = np.tanh(z1)
    z2 = np.dot(a1, w2) + b2
    logits = z2


    probs = softmax(logits)
    print ("probs", probs)
    Y_pred =  np.argmax(probs, axis=1)

    return Y_pred

In [47]:
def get_accuracy(Y, Y_pred):
    """
    Calcualtes accuracy.
    :param Y: int(N, )
        Correct labels.
    :param Y_pred: int(N, ) | double(N, C)
        Predicted labels of shape(N, ) or (N, C) in case of one-hot vector.
    :return: double
        Accuracy.
    """
    return (Y == Y_pred).mean()
    #return (np.argmax(Y, axis=1) == Y_pred).mean() #used in case of one-hot vector and loss is Negative Likelihood.

In [48]:
class Particle:
    """
    Particle class represents a solution inside a pool(Swarm).
    """
    def __init__(self, no_dim, x_range, v_range):
        """
        Particle class constructor
        :param no_dim: int
            No of dimensions.
        :param x_range: tuple(double)
            Min and Max value(range) of dimension.
        :param v_range: tuple(double)
            Min and Max value(range) of velocity.
        """
        self.x = np.random.uniform(x_range[0], x_range[1], (no_dim, )) #particle position in each dimension...
        self.v = np.random.uniform(v_range[0], v_range[1], (no_dim, )) #particle velocity in each dimension...
        self.pbest = np.inf
        self.pbestpos = np.zeros((no_dim, ))

In [49]:
class Swarm:
    """
    Swarm class represents a pool of solution(particle).
    """
    def __init__(self, no_particle, no_dim, x_range, v_range, iw_range, c):
        """
        Swarm class constructor.
        :param no_particle: int
            No of particles(solutions).
        :param no_dim: int
            No of dimensions.
        :param x_range: tuple(double)
            Min and Max value(range) of dimension.
        :param v_range: tuple(double)
            Min and Max value(range) of velocity.
        :param iw_range: tuple(double)
            Min and Max value(range) of interia weight.
        :param c: tuple(double)
            c[0] -> cognitive parameter, c[1] -> social parameter.
        """
        self.p = np.array([Particle(no_dim, x_range, v_range) for i in range(no_particle)])
        self.gbest = np.inf
        self.gbestpos = np.zeros((no_dim, ))
        self.x_range = x_range
        self.v_range = v_range
        self.iw_range = iw_range
        self.c0 = c[0]
        self.c1 = c[1]
        self.no_dim = no_dim

    def optimize(self, function, X, Y,  print_step,  iter):
        """
        optimize is used start optimization.

        :param function: function
            Function to be optimized.
        :param X: input
            Used in forward pass.
        :param Y: target
            Used to calculate loss.
        :param print_step: int
            Print pause between two adjacent prints.
        :param iter: int
            No of iterations.
        """
        for i in range(iter):
            for particle in self.p:
                fitness = function(X, Y, particle.x)

                if fitness < particle.pbest:
                    particle.pbest = fitness
                    particle.pbestpos = particle.x.copy()

                if fitness < self.gbest:
                    self.gbest = fitness
                    self.gbestpos = particle.x.copy()


            for particle in self.p:
                #Here iw is inertia weight...
                iw = np.random.uniform(self.iw_range[0], self.iw_range[1], 1)[0]
                particle.v = iw * particle.v + (self.c0 * np.random.uniform(0.0, 1.0, (self.no_dim, )) * \
                (particle.pbestpos - particle.x)) + (self.c1 * np.random.uniform(0.0, 1.0, (self.no_dim, )) \
                * (self.gbestpos - particle.x))
                #particle.v = particle.v.clip(min=self.v_range[0], max=self.v_range[1])
                particle.x = particle.x + particle.v
                #particle.x = particle.x.clip(min=self.x_range[0], max=self.x_range[1])

            if i % print_step == 0:
                print('iteration#: ', i+1,  ' loss: ', fitness)

        print("global best loss: ", self.gbest)

    def get_best_solution(self):
        '''

        :return: array of parameters/weights.
        '''
        return self.gbestpos


In [66]:
if __name__ == '__main__':
    no_solution = 2
    no_dim = (INPUT_NODES * HIDDEN_NODES) + HIDDEN_NODES + (HIDDEN_NODES * OUTPUT_NODES) + OUTPUT_NODES
    w_range = (0.0, 1.0)
    lr_range = (0.0, 1.0)
    iw_range = (0.9, 0.9)  # iw -> inertial weight...
    c = (0.5, 0.3)  # c[0] -> cognitive factor, c[1] -> social factor...

    s = Swarm(no_solution, no_dim, w_range, lr_range, iw_range, c)
    #Y = one_hot_encode(Y) #Encode here...
    s.optimize(forward_pass, X, Y, 100, 2)
    W = s.get_best_solution()
    print("X",X)
    Y_pred = predict(X, W)
    print("Y_pred",Y_pred)
    accuracy = get_accuracy(Y, Y_pred)
    print("Accuracy: %.3f"% accuracy)

Negative_Likelihood(probs, Y) 0.64934128174379
Negative_Likelihood(probs, Y) 1.0155512357273155
iteration#:  1  loss:  1.0155512357273155
Negative_Likelihood(probs, Y) 0.5528318033107118
Negative_Likelihood(probs, Y) 0.9375993405325559
global best loss:  0.5528318033107118
X        3.6216    8.6661  -2.8073  -0.44699
0     4.54590   8.16740  -2.4586  -1.46210
1     3.86600  -2.63830   1.9242   0.10645
2     3.45660   9.52280  -4.0112  -3.59440
3     0.32924  -4.45520   4.5718  -0.98880
4     4.36840   9.67180  -3.9606  -3.16250
...       ...       ...      ...       ...
1366  0.40614   1.34920  -1.4501  -0.55949
1367 -1.38870  -4.87730   6.4774   0.34179
1368 -3.75030 -13.45860  17.5932  -2.77710
1369 -3.56370  -8.38270  12.3930  -1.28230
1370 -2.54190  -0.65804   2.6842   1.19520

[1371 rows x 4 columns]
probs [[0.52841059 0.47158941]
 [0.59650486 0.40349514]
 [0.46447609 0.53552391]
 ...
 [0.32530233 0.67469767]
 [0.40756488 0.59243512]
 [0.12284282 0.87715718]]
Y_pred [0 0 1 ... 1 1